## Yeşil Alan Optimizasyonu Projesi: Veri Görselleştirme ve Analiz

Bu notebook, İstanbul ilçeleri için yeşil alan optimizasyonu projesinin sonuçlarını içeren `optimum_yesil_alan_sonuclari.csv` dosyasındaki verileri kullanarak çeşitli görselleştirmeler ve karşılaştırmalar yapmayı amaçlamaktadır.

Kullanılan temel metrikler şunlardır:
- Mevcut yeşil alan (`alan_metrekare`)
- Nüfus (`Nufus`)
- Ulaşım olanakları (Minibüs durakları, Raylı sistem istasyonları, Taksi durakları)
- Ortalama Hava Kalitesi İndeksi (`Ortalama_AQI`)
- Proje kapsamında hesaplanan Ulaşım Skoru ($T_i$) ve Hava Kalitesi Skoru ($S_i$)
- Genetik Algoritma (GA) ile eklenen ve toplam yeşil alan (`GA_Yeni_Yesil_Alan`, `GA_Toplam_Yesil_Alan`)
- Parçacık Sürü Optimizasyonu (PSO) ile eklenen ve toplam yeşil alan (`PSO_Yeni_Yesil_Alan`, `PSO_Toplam_Yesil_Alan`)

**Proje Kısıtlarından Bazıları (Referans Amaçlı):**
1. Her bir ilçe için maksimum yeşil alan büyüklüğü: $x_i \le A_{max,i}$
2. Her bir ilçenin nüfusuna göre belirlenen kişi başı minimum yeşil alan: $\frac{GA_i+x_i}{P_i} \ge \delta$
3. Pozitif yeşil alan: $x_i \ge 0$

Burada $x_i$ ilçeye eklenen yeni yeşil alanı, $GA_i$ mevcut yeşil alanı, $P_i$ nüfusu ve $\delta$ kişi başı hedeflenen minimum yeşil alan miktarını temsil eder. $A_{max,i}$ verimizde bulunmamaktadır.

### 1. Kütüphanelerin Yüklenmesi ve Veri Setinin Okunması

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Matplotlib ayarları ve Türkçe karakterler için
plt.rcParams['font.family'] = 'DejaVu Sans' # veya sisteminizde yüklü Türkçe karakter destekleyen bir font
plt.rcParams['axes.unicode_minus'] = False # Eksi işaretlerinin doğru gösterimi için

# Genel grafik ayarları
sns.set_style("whitegrid")

# Veri setini yükle
file_path = 'optimum_yesil_alan_sonuclari.csv'
try:
    df = pd.read_csv(file_path)
    print("CSV dosyası başarıyla yüklendi.")
except FileNotFoundError:
    print(f"Hata: '{file_path}' dosyası bulunamadı. Lütfen dosya yolunu kontrol edin.")
except Exception as e:
    print(f"Dosya okunurken bir hata oluştu: {e}")

### 2. Veri Setine Genel Bakış

In [ ]:
if 'df' in locals():
    print("Veri setinin ilk 5 satırı:")
    display(df.head())
    print("\nVeri seti hakkında bilgiler:")
    df.info()
    print("\nVeri setindeki sayısal sütunlar için temel istatistikler:")
    display(df.describe())
    print("\nEksik değerlerin kontrolü:")
    print(df.isnull().sum())

### 3. Görselleştirmeler

Bu bölümde, veri setindeki farklı değişkenleri ve optimizasyon sonuçlarını görselleştireceğiz.

#### 3.1. Mevcut Yeşil Alanların İlçelere Göre Dağılımı

Her bir ilçedeki mevcut yeşil alan miktarını ($m^2$ cinsinden) gösterir. Bu, optimizasyon öncesi durumu anlamak için temel bir görselleştirmedir.

In [ ]:
if 'df' in locals():
    df_sorted_mevcut = df.sort_values('alan_metrekare', ascending=False)
    plt.figure(figsize=(16, 9))
    sns.barplot(x='alan_metrekare', y='ILCE', data=df_sorted_mevcut, palette='viridis')
    plt.title('Mevcut Yeşil Alanların İlçelere Göre Dağılımı ($m^2$)', fontsize=16)
    plt.xlabel('Mevcut Yeşil Alan ($m^2$)', fontsize=12)
    plt.ylabel('İlçe', fontsize=12)
    plt.tight_layout()
    plt.show()

#### 3.2. Optimizasyon Yöntemlerine Göre Eklenen Yeşil Alan Miktarları

Genetik Algoritma (GA) ve Parçacık Sürü Optimizasyonu (PSO) yöntemleriyle her bir ilçeye ne kadar yeni yeşil alan ($m^2$) eklendiğini karşılaştırmalı olarak gösterir.

In [ ]:
if 'df' in locals():
    df_melted_yeni = df.melt(id_vars='ILCE', value_vars=['GA_Yeni_Yesil_Alan', 'PSO_Yeni_Yesil_Alan'],
                               var_name='Optimizasyon Yöntemi', value_name='Eklenen Yeşil Alan ($m^2$)')
    df_melted_yeni['Optimizasyon Yöntemi'] = df_melted_yeni['Optimizasyon Yöntemi'].map({
        'GA_Yeni_Yesil_Alan': 'Genetik Algoritma (GA)',
        'PSO_Yeni_Yesil_Alan': 'Parçacık Sürü Optimizasyonu (PSO)'
    })

    # İlçeleri ortalama eklenen alana göre sıralayarak daha okunaklı bir grafik elde edelim
    ilce_order_yeni = df.groupby('ILCE')[['GA_Yeni_Yesil_Alan', 'PSO_Yeni_Yesil_Alan']].mean().mean(axis=1).sort_values(ascending=False).index

    plt.figure(figsize=(18, 12))
    sns.barplot(x='Eklenen Yeşil Alan ($m^2$)', y='ILCE', hue='Optimizasyon Yöntemi', 
                data=df_melted_yeni, palette={'Genetik Algoritma (GA)': 'coral', 'Parçacık Sürü Optimizasyonu (PSO)': 'skyblue'},
                order=ilce_order_yeni)
    plt.title('Optimizasyon Yöntemlerine Göre İlçelere Eklenen Yeşil Alan Miktarları ($m^2$)', fontsize=16)
    plt.xlabel('Eklenen Yeşil Alan ($m^2$)', fontsize=12)
    plt.ylabel('İlçe', fontsize=12)
    plt.legend(title='Optimizasyon Yöntemi')
    plt.tight_layout()
    plt.show()

#### 3.3. Mevcut ve Optimizasyon Sonrası Toplam Yeşil Alanların Karşılaştırılması

Her ilçe için mevcut yeşil alan miktarını, GA optimizasyonu sonrası toplam yeşil alan miktarını ve PSO optimizasyonu sonrası toplam yeşil alan miktarını ($m^2$) yan yana gösterir.

In [ ]:
if 'df' in locals():
    df_toplam_melted = df.melt(id_vars='ILCE',
                               value_vars=['alan_metrekare', 'GA_Toplam_Yesil_Alan', 'PSO_Toplam_Yesil_Alan'],
                               var_name='Yeşil Alan Türü',
                               value_name='Alan ($m^2$)')
    df_toplam_melted['Yeşil Alan Türü'] = df_toplam_melted['Yeşil Alan Türü'].map({
        'alan_metrekare': 'Mevcut Yeşil Alan',
        'GA_Toplam_Yesil_Alan': 'Toplam Yeşil Alan (GA)',
        'PSO_Toplam_Yesil_Alan': 'Toplam Yeşil Alan (PSO)'
    })

    # İlçeleri mevcut yeşil alana göre sırala
    ilce_order_toplam = df.sort_values('alan_metrekare', ascending=False)['ILCE']

    plt.figure(figsize=(20, 15))
    sns.barplot(x='Alan ($m^2$)', y='ILCE', hue='Yeşil Alan Türü', data=df_toplam_melted,
                palette={'Mevcut Yeşil Alan': 'lightgreen', 'Toplam Yeşil Alan (GA)': 'salmon', 'Toplam Yeşil Alan (PSO)': 'lightblue'},
                order=ilce_order_toplam)
    plt.title('Mevcut ve Optimizasyon Sonrası Toplam Yeşil Alanların Karşılaştırılması ($m^2$)', fontsize=16)
    plt.xlabel('Yeşil Alan ($m^2$)', fontsize=12)
    plt.ylabel('İlçe', fontsize=12)
    plt.legend(title='Yeşil Alan Durumu')
    plt.tight_layout()
    plt.show()

#### 3.4. Kişi Başına Düşen Toplam Yeşil Alan (GA vs PSO)

GA ve PSO optimizasyonları sonucunda her bir ilçede kişi başına düşen toplam yeşil alan miktarını ($m^2$/kişi) karşılaştırır. Grafikte örnek olarak 10 $m^2$/kişi ve 15 $m^2$/kişi gibi hedefler de gösterilmiştir. Bu, $\frac{GA_i+x_i}{P_i} \ge \delta$ kısıtının sonuçlarını değerlendirmek için önemlidir.

In [ ]:
if 'df' in locals():
    df['Kişi Başı GA ($m^2$)'] = df['GA_Toplam_Yesil_Alan'] / df['Nufus']
    df['Kişi Başı PSO ($m^2$)'] = df['PSO_Toplam_Yesil_Alan'] / df['Nufus']

    df_kisi_basi_melted = df.melt(id_vars='ILCE', value_vars=['Kişi Başı GA ($m^2$)', 'Kişi Başı PSO ($m^2$)'],
                                   var_name='Hesaplama Yöntemi', value_name='Kişi Başı Yeşil Alan ($m^2$/kişi)')
    df_kisi_basi_melted['Hesaplama Yöntemi'] = df_kisi_basi_melted['Hesaplama Yöntemi'].map({
        'Kişi Başı GA ($m^2$)': 'GA ile Optimize Edilmiş',
        'Kişi Başı PSO ($m^2$)': 'PSO ile Optimize Edilmiş'
    })

    # İlçeleri ortalama kişi başı yeşil alana göre sırala
    ilce_order_kisi_basi = df.groupby('ILCE')[['Kişi Başı GA ($m^2$)', 'Kişi Başı PSO ($m^2$)']].mean().mean(axis=1).sort_values(ascending=False).index

    plt.figure(figsize=(18, 12))
    sns.barplot(x='Kişi Başı Yeşil Alan ($m^2$/kişi)', y='ILCE', hue='Hesaplama Yöntemi', data=df_kisi_basi_melted,
                palette={'GA ile Optimize Edilmiş': 'mediumseagreen', 'PSO ile Optimize Edilmiş': 'cornflowerblue'},
                order=ilce_order_kisi_basi)
    plt.title('Optimizasyon Sonrası Kişi Başına Düşen Toplam Yeşil Alan ($m^2$/kişi)', fontsize=16)
    plt.xlabel('Kişi Başına Düşen Yeşil Alan ($m^2$/kişi)', fontsize=12)
    plt.ylabel('İlçe', fontsize=12)
    plt.axvline(x=10, color='red', linestyle='--', label='Hedef $\delta_1$: 10 $m^2$/kişi') # Örnek bir hedef çizgi
    plt.axvline(x=15, color='darkred', linestyle='--', label='Hedef $\delta_2$: 15 $m^2$/kişi') # Örnek bir hedef çizgi
    plt.legend(title='Optimizasyon Yöntemi ve Hedefler')
    plt.tight_layout()
    plt.show()

#### 3.5. Eklenen Yeşil Alan ve Ortalama Hava Kalitesi İndeksi (AQI) İlişkisi

Her bir ilçe için optimizasyonla eklenen yeşil alan miktarı ($m^2$) ile o ilçenin ortalama Hava Kalitesi İndeksi (AQI) arasındaki ilişkiyi gösterir. AQI'da düşük değerler daha iyi hava kalitesini ifade eder.

In [ ]:
if 'df' in locals():
    plt.figure(figsize=(15, 7))

    plt.subplot(1, 2, 1)
    sns.scatterplot(x='Ortalama_AQI', y='GA_Yeni_Yesil_Alan', data=df, color='coral', s=100, alpha=0.7, legend=False)
    # Noktaların üzerine ilçe isimlerini yazmak için (isteğe bağlı, kalabalık olabilir):
    # for i in range(df.shape[0]):
    #     plt.text(df.Ortalama_AQI[i], df.GA_Yeni_Yesil_Alan[i], df.ILCE[i], fontsize=7)
    plt.title('GA: Eklenen Yeşil Alan vs. Ortalama AQI', fontsize=14)
    plt.xlabel('Ortalama Hava Kalitesi İndeksi (AQI)\n(Düşük değer daha iyi hava kalitesi)', fontsize=10)
    plt.ylabel('GA ile Eklenen Yeşil Alan ($m^2$)', fontsize=10)
    plt.grid(True)

    plt.subplot(1, 2, 2)
    sns.scatterplot(x='Ortalama_AQI', y='PSO_Yeni_Yesil_Alan', data=df, color='skyblue', s=100, alpha=0.7, legend=False)
    # for i in range(df.shape[0]):
    #     plt.text(df.Ortalama_AQI[i], df.PSO_Yeni_Yesil_Alan[i], df.ILCE[i], fontsize=7)
    plt.title('PSO: Eklenen Yeşil Alan vs. Ortalama AQI', fontsize=14)
    plt.xlabel('Ortalama Hava Kalitesi İndeksi (AQI)\n(Düşük değer daha iyi hava kalitesi)', fontsize=10)
    plt.ylabel('PSO ile Eklenen Yeşil Alan ($m^2$)', fontsize=10)
    plt.grid(True)

    plt.tight_layout()
    plt.show()

#### 3.6. Eklenen Yeşil Alan ve Ulaşım Skoru ($T_i$) İlişkisi

Her bir ilçe için optimizasyonla eklenen yeşil alan miktarı ($m^2$) ile o ilçenin ulaşım skoru ($T_i$) arasındaki ilişkiyi gösterir. Ulaşım skorunda yüksek değerler daha iyi ulaşım imkanlarını ifade eder.

In [ ]:
if 'df' in locals():
    plt.figure(figsize=(15, 7))

    plt.subplot(1, 2, 1)
    sns.scatterplot(x='Ti', y='GA_Yeni_Yesil_Alan', data=df, color='coral', s=100, alpha=0.7, legend=False)
    plt.title('GA: Eklenen Yeşil Alan vs. Ulaşım Skoru ($T_i$)', fontsize=14)
    plt.xlabel('Ulaşım Skoru ($T_i$)\n(Yüksek değer daha iyi ulaşım)', fontsize=10)
    plt.ylabel('GA ile Eklenen Yeşil Alan ($m^2$)', fontsize=10)
    plt.grid(True)

    plt.subplot(1, 2, 2)
    sns.scatterplot(x='Ti', y='PSO_Yeni_Yesil_Alan', data=df, color='skyblue', s=100, alpha=0.7, legend=False)
    plt.title('PSO: Eklenen Yeşil Alan vs. Ulaşım Skoru ($T_i$)', fontsize=14)
    plt.xlabel('Ulaşım Skoru ($T_i$)\n(Yüksek değer daha iyi ulaşım)', fontsize=10)
    plt.ylabel('PSO ile Eklenen Yeşil Alan ($m^2$)', fontsize=10)
    plt.grid(True)

    plt.tight_layout()
    plt.show()

#### 3.7. GA vs. PSO ile Eklenen Yeşil Alan Miktarları (İlçe Bazlı Fark Karşılaştırması)

Her bir ilçe için PSO ile eklenen yeşil alan miktarı ile GA ile eklenen yeşil alan miktarı arasındaki farkı ($m^2$) gösterir. Pozitif değerler PSO'nun o ilçede GA'dan daha fazla alan önerdiğini, negatif değerler ise GA'nın daha fazla alan önerdiğini belirtir.

In [ ]:
if 'df' in locals():
    df_comparison = df[['ILCE', 'GA_Yeni_Yesil_Alan', 'PSO_Yeni_Yesil_Alan']].copy()
    df_comparison['Fark (PSO - GA) ($m^2$)'] = df_comparison['PSO_Yeni_Yesil_Alan'] - df_comparison['GA_Yeni_Yesil_Alan']
    df_comparison_sorted = df_comparison.sort_values('Fark (PSO - GA) ($m^2$)', ascending=False)

    plt.figure(figsize=(16, 10))
    sns.barplot(x='Fark (PSO - GA) ($m^2$)', y='ILCE', data=df_comparison_sorted, palette='coolwarm')
    plt.title('PSO ve GA ile Eklenen Yeşil Alan Miktarları Arasındaki Fark ($m^2$)\n(Pozitif: PSO > GA, Negatif: GA > PSO)', fontsize=16)
    plt.xlabel('Fark: PSO Eklenen Alan - GA Eklenen Alan ($m^2$)', fontsize=12)
    plt.ylabel('İlçe', fontsize=12)
    plt.axvline(0, color='black', lw=0.8)
    plt.tight_layout()
    plt.show()

#### 3.8. Temel Metriklerin Dağılımları (Ortalama AQI, Ulaşım Skoru ($T_i$), Hava Kalitesi Skoru ($S_i$))

İlçeler genelinde Ortalama AQI, Ulaşım Skoru ($T_i$) ve Hava Kalitesi Skoru ($S_i$) değerlerinin dağılımlarını gösterir. Bu, bu metriklerin genel yapısını anlamak için faydalıdır.

In [ ]:
if 'df' in locals():
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    
    sns.histplot(df['Ortalama_AQI'], kde=True, ax=axes[0], color='skyblue', bins=10)
    axes[0].set_title('Ortalama AQI Dağılımı', fontsize=14)
    axes[0].set_xlabel('Ortalama AQI', fontsize=10)
    axes[0].set_ylabel('Frekans', fontsize=10)

    sns.histplot(df['Ti'], kde=True, ax=axes[1], color='lightcoral', bins=10)
    axes[1].set_title('Ulaşım Skoru ($T_i$) Dağılımı', fontsize=14)
    axes[1].set_xlabel('Ulaşım Skoru ($T_i$)', fontsize=10)
    axes[1].set_ylabel('Frekans', fontsize=10)

    sns.histplot(df['Si'], kde=True, ax=axes[2], color='lightgreen', bins=10)
    axes[2].set_title('Hava Kalitesi Skoru ($S_i$) Dağılımı', fontsize=14)
    axes[2].set_xlabel('Hava Kalitesi Skoru ($S_i$)', fontsize=10)
    axes[2].set_ylabel('Frekans', fontsize=10)

    plt.tight_layout()
    plt.show()

### 4. Analizli Veri Setinin Kaydedilmesi

Hesaplanan kişi başı yeşil alan miktarları gibi ek bilgilerle güncellenmiş veri setini yeni bir CSV dosyasına kaydedelim.

In [ ]:
if 'df' in locals():
    # Kişi başı alan sütunlarının varlığını kontrol et, yoksa oluştur (önceki hücrelerde oluşturulmuş olmalı)
    if 'Kişi Başı GA ($m^2$)' not in df.columns:
        df['Kişi Başı GA ($m^2$)'] = df['GA_Toplam_Yesil_Alan'] / df['Nufus']
    if 'Kişi Başı PSO ($m^2$)' not in df.columns:
        df['Kişi Başı PSO ($m^2$)'] = df['PSO_Toplam_Yesil_Alan'] / df['Nufus']
        
    output_csv_path = 'optimum_yesil_alan_sonuclari_analizli.csv'
    try:
        df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
        print(f"\nAnalizli veri seti '{output_csv_path}' dosyasına başarıyla kaydedildi.")
    except Exception as e:
        print(f"Analizli veri seti kaydedilirken bir hata oluştu: {e}")

### 5. Sonuç

Bu notebook, sağlanan CSV dosyasındaki verileri kullanarak çeşitli karşılaştırmalı grafikler üretmiştir. Bu görselleştirmeler, Genetik Algoritma ve Parçacık Sürü Optimizasyonu yöntemlerinin İstanbul'daki yeşil alan dağılımını nasıl etkilediğini, mevcut durumla kıyaslamalarını, kişi başına düşen yeşil alan miktarlarını ve çeşitli çevresel/ulaşım faktörleriyle ilişkilerini ortaya koymaktadır. 

Elde edilen grafikler ve analizli veri seti, projenizin bulgularını daha etkili bir şekilde sunmanıza yardımcı olacaktır.